# MAS 649 Midterm
# Group 5
### Austin Gravely, Elias Eskind, Arlet Rodriguez, Cesar Diez, Juliano Torii
### 4/6/21

# Step 1: Loading & Splitting Data

## Loading in Data

In [ ]:
btc = spark.read.csv('/FileStore/tables/BTC_USD.csv',inferSchema=True,header=True)
eth = spark.read.csv('/FileStore/tables/ETH_USD.csv',inferSchema=True,header=True)
theta = spark.read.csv('/FileStore/tables/THETA_USD.csv',inferSchema=True,header=True)

## Viewing Data

In [ ]:
# Display
display(btc)
display(eth)
display(theta)

In [ ]:
# View Schema
btc.printSchema()
eth.printSchema()
theta.printSchema()

## Dropping N/A's

In [ ]:
btc2=btc.na.drop()
eth2=eth.na.drop()
theta2=theta.na.drop()

## Changing 'Date' Column from String to Date Datatype

In [ ]:
from pyspark.sql.functions import to_date

# Bitcoin
btc2 = btc2.withColumn("Date", to_date(btc2['Date'], "M/d/yy"))
btc2.printSchema()

# Ethereum
eth2 = eth2.withColumn("Date", to_date(eth2['Date'], "M/d/yy"))
eth2.printSchema()

# Theta
theta2 = theta2.withColumn("Date", to_date(theta2['Date'], "M/d/yy"))
theta2.printSchema()

root
-- Date: date (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: long (nullable = true)
-- ReturnY: double (nullable = true)
-- OCDiff: double (nullable = true)
-- Lag1: double (nullable = true)
-- Lag2: double (nullable = true)
-- Lag3: double (nullable = true)
-- Lag4: double (nullable = true)

root
-- Date: date (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: long (nullable = true)
-- ReturnY: double (nullable = true)
-- OCDiff: double (nullable = true)
-- Lag1: double (nullable = true)
-- Lag2: double (nullable = true)
-- Lag3: double (nullable = true)
-- Lag4: double (nullable = true)

root
-- Date: date (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: integer (nullable = true)
-- ReturnY: double (nullable = true)
-- OCDiff: double (nullable = true)
-- Lag1: double (nullable = true)
-- Lag2: double (nullable = true)
-- Lag3: double (nullable = true)
-- Lag4: double (nullable = true)

## Splitting data into Train & Test

### Bitcoin

In [ ]:
# Train
train_databtc2 = btc2.limit(316)
display(train_databtc2)

#Test
test_databtc2 = btc2.orderBy(btc2["Date"].desc()).limit(30).orderBy("Date")
display(test_databtc2)

### Ethereum

In [ ]:
# Train
train_dataeth2 = eth2.limit(316)
display(train_dataeth2)

#Test
test_dataeth2 = eth2.orderBy(eth2["Date"].desc()).limit(30).orderBy("Date")
display(test_dataeth2)

### Theta

In [ ]:
# Train
train_datatheta2 = theta2.limit(316)
display(train_datatheta2)

#Test
test_datatheta2 = theta2.orderBy(theta2["Date"].desc()).limit(30).orderBy("Date")
display(test_datatheta2)

## Transforming Data

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'OCDiff',
 'Lag1',
 'Lag2',
 'Lag3',
 'Lag4'], outputCol = "features")

In [ ]:
# Bitcoin
outputTrainbtc2 = assembler.transform(train_databtc2)
display(outputTrainbtc2)

outputTestbtc2 = assembler.transform(test_databtc2)
display(outputTestbtc2)

# Ethereum
outputTraineth2 = assembler.transform(train_dataeth2)
display(outputTraineth2)

outputTesteth2  = assembler.transform(test_dataeth2)
display(outputTesteth2)

# Theta
outputTraintheta2 = assembler.transform(train_datatheta2)
display(outputTraintheta2)

outputTesttheta2  = assembler.transform(test_datatheta2)
display(outputTesttheta2)

## Final Datasets

### Bitcoin

In [ ]:
# Final Train
trainBtc = outputTrainbtc2.select("features", "ReturnY", "Date")
display(trainBtc)

# Final Test
testBtc = outputTestbtc2.select("features", "ReturnY", "Date")
display(testBtc)

# Displaying
display(trainBtc.describe())
display(testBtc.describe())

### Ethereum

In [ ]:
#Final Train
trainEth = outputTraineth2.select("features", "ReturnY", "Date")
display(trainEth)

#Final Test
testEth = outputTesteth2.select("features", "ReturnY", "Date")
display(testEth)

# Displaying
display(trainEth.describe())
display(testEth.describe())

### Theta

In [ ]:
#Final Train 
trainTheta = outputTraintheta2.select("features", "ReturnY", "Date")
display(trainTheta)

#Final Test 
testTheta = outputTesttheta2.select("features", "ReturnY", "Date")
display(testTheta)

# Displaying
display(trainTheta.describe())
display(testTheta.describe())

# Step 2: ML Algorithm

## Importing Required Packages

In [ ]:
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor, DecisionTreeRegressor, LinearRegression

In [ ]:
dt = DecisionTreeRegressor(labelCol = "ReturnY", featuresCol = "features")
rf = RandomForestRegressor(labelCol = "ReturnY", featuresCol = "features")
gb = GBTRegressor(labelCol = "ReturnY", featuresCol = "features")

## Training Models

In [ ]:
# Bitcoin
dt_modelBtc = dt.fit(trainBtc)
rf_modelBtc = rf.fit(trainBtc)
gb_modelBtc = gb.fit(trainBtc)

# Ethereum
dt_modelEth = dt.fit(trainEth)
rf_modelEth = rf.fit(trainEth)
gb_modelEth = gb.fit(trainEth)

# Theta
dt_modelTheta = dt.fit(trainTheta)
rf_modelTheta = rf.fit(trainTheta)
gb_modelTheta = gb.fit(trainTheta)

## Testing Models

In [ ]:
# Bitcoin
dt_predictionsBtc = dt_modelBtc.transform(testBtc)
rf_predictionsBtc = rf_modelBtc.transform(testBtc)
gb_predictionsBtc = gb_modelBtc.transform(testBtc)

# Ethereum
dt_predictionsEth = dt_modelEth.transform(testEth)
rf_predictionsEth = rf_modelEth.transform(testEth)
gb_predictionsEth = gb_modelEth.transform(testEth)

# Theta
dt_predictionsTheta = dt_modelTheta.transform(testTheta)
rf_predictionsTheta = rf_modelTheta.transform(testTheta)
gb_predictionsTheta = gb_modelTheta.transform(testTheta)

## Evaluator

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="ReturnY", predictionCol="prediction", metricName="rmse")

# Bitcoin
print('Bitcoin:')
print(evaluator.evaluate(dt_predictionsBtc))
print(evaluator.evaluate(rf_predictionsBtc))
print(evaluator.evaluate(gb_predictionsBtc))

# Ethereum
print('Ethereum:')
print(evaluator.evaluate(dt_predictionsEth))
print(evaluator.evaluate(rf_predictionsEth))
print(evaluator.evaluate(gb_predictionsEth))

# Theta
print('Theta:')
print(evaluator.evaluate(dt_predictionsTheta))
print(evaluator.evaluate(rf_predictionsTheta))
print(evaluator.evaluate(gb_predictionsTheta))

Bitcoin:
0.035215985893660956
0.03640528574677352
0.03828398855111902
Ethereum:
0.06137721727207424
0.04927536011320538
0.0679509252807517
Theta:
0.055323597787211444
0.048447958485873376
0.05667084581327231

## Hyperparameter RF

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

paramGrid = ParamGridBuilder().addGrid(rf.maxDepth, [2,4,6]).addGrid(rf.maxBins, [20,60]).build()

crossval = CrossValidator(estimator = rf, estimatorParamMaps=paramGrid, evaluator = evaluator, numFolds = 10)

In [ ]:
# Training Models
cv_modelTheta = crossval.fit(trainTheta)
cv_modelEth = crossval.fit(trainEth)
cv_modelBtc = crossval.fit(trainBtc)

# RMSE:
# Bitcoin
print('Bitcoin:')
cv_predictionsBtc = cv_modelBtc.transform(testBtc)
print(evaluator.evaluate(cv_predictionsBtc))

# Ethereum
print('Ethereum:')
cv_predictionsEth = cv_modelEth.transform(testEth)
print(evaluator.evaluate(cv_predictionsEth))

# Theta
print('Theta:')
cv_predictionsTheta = cv_modelTheta.transform(testTheta)
print(evaluator.evaluate(cv_predictionsTheta))

# You may need to install mlflow if this chunk is taking forever
# Even so this chunk will take 5 minutes or so

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
Bitcoin:
0.03839393151577049
Ethereum:
0.04895113745563109
Theta:
0.05044714477677209

## Elastic Net Hyperparameter Tuning

In [ ]:
# Importing Packages
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(labelCol="ReturnY", featuresCol="features")
lrModel = lr.fit(trainTheta)
lrModel

# Training Models
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.1, 0.01]).addGrid(lr.elasticNetParam, [0, 0.5, 1]).build()
crossval = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=10)
cv_modelTheta = crossval.fit(trainTheta)
cv_modelEth = crossval.fit(trainEth)
cv_modelBtc = crossval.fit(trainBtc)

# RMSE:
# Bitcoin
print('Bitcoin:')
cv2_predictionsBtc = cv_modelBtc.transform(testBtc)
print(evaluator.evaluate(cv2_predictionsBtc))

# Ethereum
print('Ethereum:')
cv2_predictionsEth = cv_modelEth.transform(testEth)
print(evaluator.evaluate(cv2_predictionsEth))

# Theta
print('Theta:')
cv2_predictionsTheta = cv_modelTheta.transform(testTheta)
print(evaluator.evaluate(cv2_predictionsTheta))

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
Bitcoin:
0.0345924050724973
Ethereum:
0.048332895468184395
Theta:
0.04896190875934517

In [ ]:
# import numpy as np
# print(cv_model.getEstimatorParamMaps()[np.argmax(cv_model.avgMetrics)])
# print(cv_model.bestModel._java_obj.getRegParam())
# print(cv_model.bestModel._java_obj.getElasticNetParam())

## Basic Linear Regression (for Ethereum since this Model had the Best RMSE)

In [ ]:
from pyspark.ml.regression import LinearRegression
ethlr = LinearRegression(labelCol = "ReturnY", featuresCol = "features", solver = 'l-bfgs', regParam = 0.2, elasticNetParam = 0.6)

lrModelEth = ethlr.fit(trainEth)
# printing coeff and intercept
print("Coefficients: %s" % str(lrModelEth.coefficients))
print("Intercept: %s" % str(lrModelEth.intercept))

# more details about model
trainingSummary = lrModelEth.summary
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: (10,[],[])
Intercept: 0.004585358183544301
+--------------------+
 residuals|
+--------------------+
-0.00286519818354...|
 -0.0709722611835443|
-0.08698793318354431|
 0.08096090881645569|
-0.03291953718354...|
 -0.0065578201835443|
 -0.0118305161835443|
0.028517978816455694|
 -0.0329781331835443|
-0.00108808218354...|
 -0.0248021131835443|
-0.00532408118354...|
0.002168184816455699|
0.004125127816455699|
 0.0455637708164557|
 -0.0203506281835443|
-0.02737469418354...|
0.004604820816455699|
 -0.0305163301835443|
 0.0484836198164557|
+--------------------+
only showing top 20 rows

RMSE: 0.050296
r2: 0.000000

In [ ]:
#RMSE
testResultsEth = lrModelEth.evaluate(testEth)
rmseEth = testResultsEth.rootMeanSquaredError
rmseEth

Out[25]: 0.04678414493012603

# Step 3: Testing Best Models on Dataset

## Bitcoin

In [ ]:
# Bitcoin Model with Lowest RMSE = ELASTIC NET HYPERPARAMETER TURNING
cv2_predictionsBtc = cv_modelBtc.transform(testBtc)
rmseBtc = evaluator.evaluate(cv2_predictionsBtc)
rmseBtc

Out[26]: 0.0345924050724973

## Ethereum

In [ ]:
# Ethereum Model with Lowest RMSE = LINEAR REGRESSION
testResultsEth = lrModelEth.evaluate(testEth)
rmseEth = testResultsEth.rootMeanSquaredError
rmseEth

Out[27]: 0.04678414493012603

## Theta

In [ ]:
# Theta Model with Lowest RMSE = RANDOM FOREST
rf_predictionsTheta = rf_modelTheta.transform(testTheta)
rmseTheta = evaluator.evaluate(rf_predictionsTheta)
rmseTheta

Out[28]: 0.048447958485873376

# Step 4: Average the RMSE

In [ ]:
import numpy as np
rmseList = [rmseBtc, rmseEth, rmseTheta]
rmseMean = np.mean(rmseList)
print(rmseMean)

0.04327483616283224

## The Avergae RMSE for BTC, ETH, & THETA is:  *0.04327*

# Step 5: Visualizations

## Ethereum Graphs

In [ ]:
print(btc2.filter(btc2["Date"] > "2020-11-09").count())
print(eth2.filter(eth2["Date"] > "2020-11-09").count())
print(theta2.filter(theta2["Date"] > "2020-11-09").count())

30
30
30

In [ ]:
display(eth2.select("ReturnY", "Date"))

In [ ]:
display(eth2.select("Volume", "Date"))

In [ ]:
display(eth2.select("Close","Low", "High", "Date"))

## Candlestick Graphs (Extra)

In [ ]:
import plotly.graph_objects as go

### Bitcoin

In [ ]:
#Converting dataframe from Pyspark to Pandas
df_btc = btc.toPandas()

fig = go.Figure(data=[go.Candlestick(x=df_btc['Date'],
              open=df_btc['Open'],
              high=df_btc['High'],
              low=df_btc['Low'],
              close=df_btc['Close'],
              increasing_line_color= 'magenta', decreasing_line_color= 'cyan')])

fig.update_layout(title='Bitcoin', yaxis_title='Price')

fig.show()

### Ethereum

In [ ]:
#Converting dataframe from Pyspark to Pandas
df_eth = eth.toPandas()

fig = go.Figure(data=[go.Candlestick(x=df_eth['Date'],
              open=df_eth['Open'],
              high=df_eth['High'],
              low=df_eth['Low'],
              close=df_eth['Close'],
              increasing_line_color= 'magenta', decreasing_line_color= 'cyan')])

fig.update_layout(title='Etheruem', yaxis_title='Price')

fig.show()

### Theta

In [ ]:
#Converting dataframe from Pyspark to Pandas
df_theta = theta.toPandas()

fig = go.Figure(data=[go.Candlestick(x=df_theta['Date'],
              open=df_theta['Open'],
              high=df_theta['High'],
              low=df_theta['Low'],
              close=df_theta['Close'],
              increasing_line_color= 'magenta', decreasing_line_color= 'cyan')])

fig.update_layout(title='Theta', yaxis_title='Price')

fig.show()

# Step 6: Options
### We chose options 1, 3, & 6 (poster is in the file)

## Option 1
##### Download (from finance.yahoo.com) and test your data on the next month of data for all three currencies.

In [ ]:
# Load in new data
btcNM = spark.read.csv('/FileStore/tables/BTC_USDNM.csv',inferSchema=True,header=True)
ethNM = spark.read.csv('/FileStore/tables/ETH_USDNM.csv',inferSchema=True,header=True)
thetaNM = spark.read.csv('/FileStore/tables/THETA_USDNM.csv',inferSchema=True,header=True)

### Pretty much Step 1 again

In [ ]:
display(btcNM)
btcNM.printSchema()
display(ethNM)
ethNM.printSchema()
display(thetaNM)
thetaNM.printSchema()

# Dropping NAs
btcNM = btcNM.na.drop()
ethNM = ethNM.na.drop()
thetaNM = thetaNM.na.drop()

# Asssembler
assembler = VectorAssembler(inputCols = ['Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'OCDiff',
 'Lag1',
 'Lag2',
 'Lag3',
 'Lag4'], outputCol = "features")

testbtcNM = assembler.transform(btcNM)
testethNM = assembler.transform(ethNM)
testthetaNM = assembler.transform(thetaNM)

# Final Test Data Bitcoin
testbtcNM2 = testbtcNM.select("features", "ReturnY", "Date")
display(testbtcNM2)

# Final Test Data Ethereum
testethNM2 = testethNM.select("features", "ReturnY", "Date")
display(testethNM2)

# Final Test Data Theta
testthetaNM2 = testthetaNM.select("features", "ReturnY", "Date")
display(testthetaNM)

### Results

#### Bitcoin

In [ ]:
#RMSE
test_resultsbtcNM = cv_modelBtc.transform(testbtcNM2)
rmse_btcNM = evaluator.evaluate(test_resultsbtcNM)
rmse_btcNM

Out[52]: 0.05066156491797875

#### Etereum

In [ ]:
#RMSE
test_resultsethNM = lrModelEth.evaluate(testethNM2)
rmse_ethNM = test_resultsethNM.rootMeanSquaredError
print(rmse_ethNM)

# Displaying residuals
display(test_resultsethNM.residuals)

#### Theta

In [ ]:
# Finding rmse
test_resultsthetaNM = rf_modelTheta.transform(testthetaNM2)
rmse_thetaNM = evaluator.evaluate(test_resultsthetaNM)
rmse_thetaNM

Out[54]: 0.10895255591091357

## Option 3

##### Find a single algorithm that works well on all of the 3 crypto-currencies and see how it works on other crypto-currencies (download another crypto currency and predict it for 30 days). You may choose your timeframes for both train and test.

### Loading in and Transforming Dogecoin Dataset

In [ ]:
#Loading in dataset
doge = spark.read.csv('/FileStore/tables/DOGE_USD.csv',inferSchema=True,header=True)
#display(doge)
doge.printSchema()

root
-- Date: string (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: long (nullable = true)
-- ReturnY: double (nullable = true)
-- OCDIff: double (nullable = true)
-- Lag1: double (nullable = true)
-- Lag2: double (nullable = true)
-- Lag3: double (nullable = true)
-- Lag4: double (nullable = true)

In [ ]:
# Dropping na
doge2=doge.na.drop()

# Changing the date datatype to date
from pyspark.sql.functions import to_date
doge2 = doge2.withColumn("Date", to_date(doge2['Date'], "M/d/yy"))
doge2.printSchema()

# Splitting into traingin and testing dataset
train_datadoge2 = doge2.limit(315)
test_datadoge2 = doge2.orderBy(doge2["Date"].desc()).limit(30).orderBy("Date")

root
-- Date: date (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: long (nullable = true)
-- ReturnY: double (nullable = true)
-- OCDIff: double (nullable = true)
-- Lag1: double (nullable = true)
-- Lag2: double (nullable = true)
-- Lag3: double (nullable = true)
-- Lag4: double (nullable = true)

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'OCDIff',
 'Lag1',
 'Lag2',
 'Lag3',
 'Lag4'], outputCol = "features")

outputTraindoge2 = assembler.transform(train_datadoge2)
outputTestdoge2 = assembler.transform(test_datadoge2)

#Final Train DOGE
trainDoge = outputTraindoge2.select("features", "ReturnY", "Date")
#Final Test DOGE
testDoge = outputTestdoge2.select("features", "ReturnY", "Date")

# Example
display(trainDoge)

In [ ]:
from pyspark.ml.regression import LinearRegression

dogeLR = LinearRegression(labelCol = "ReturnY", featuresCol = "features", solver = 'l-bfgs', regParam=0.7)

lrModelDoge = dogeLR.fit(trainDoge)
# printing coeff and intercept
print("Coefficients: %s" % str(lrModelDoge.coefficients))
print("Intercept: %s" % str(lrModelDoge.intercept))

# more details about model
trainingSummary = lrModelDoge.summary
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [-0.00872904356885888,-0.002856347070925378,-0.023427173666502356,0.006829590126776283,-6.325500156720279e-13,0.1331572256838448,0.018137690620592683,0.002616455060123739,0.005381066165874766,-0.007512860778298973]
Intercept: 0.019318547828731315
+--------------------+
 residuals|
+--------------------+
-0.01600446173661...|
-0.04503513684883108|
-0.00752012963351...|
-0.01926347316072...|
-0.03465522821098...|
-0.01658094525903293|
-0.03135388492551737|
 0.02643488489001497|
-0.00142392270021...|
0.007583026405368093|
-0.00127758509843...|
 0.1051202702880282|
-0.04387743945095175|
-0.02563065645199...|
 0.05660322632918762|
-0.04877190566982...|
0.002017571928318942|
0.002274333824202758|
-0.03405011496248288|
-0.03079969606797...|
+--------------------+
only showing top 20 rows

RMSE: 0.181384
r2: 0.010896

In [ ]:
#RMSE
testResultsDoge = lrModelDoge.evaluate(testDoge)
rmseDoge = testResultsDoge.rootMeanSquaredError
rmseDoge

Out[69]: 0.058801133560562255

### RMSE = .05880
##### If we were to guess the reason why this model's RMSE is higher than the other cryptocurrencies is because of Dogecoin's extreme volatility.

### Doge Candlestock Graph

In [ ]:
#Converting dataframe from Pyspark to Pandas
df_doge = doge.toPandas()

fig = go.Figure(data=[go.Candlestick(x=df_doge['Date'],
              open=df_doge['Open'],
              high=df_doge['High'],
              low=df_doge['Low'],
              close=df_doge['Close'],
              increasing_line_color= 'magenta', decreasing_line_color= 'cyan')])

fig.update_layout(title='Dogecoin to the Moon', yaxis_title='Price')

fig.show()

### Predicting Next 30 Days

In [ ]:
lr_predictions = lrModelDoge.transform(testDoge)
lr_predictions.select("prediction","ReturnY","features").show(30)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="ReturnY",metricName="r2")
print("R^2 = %g" % lr_evaluator.evaluate(lr_predictions))

+--------------------+------------+--------------------+
 prediction| ReturnY| features|
+--------------------+------------+--------------------+
0.017156772645292504| 0.027716745|[0.050028,0.05085...|
0.017384787570601724| 0.022320728|[0.049601,0.05239...|
0.017819175083312714| 0.188538429|[0.05098,0.052141...|
0.019341536811785555|-0.063778269|[0.052123,0.06194...|
 0.01503765038198189|-0.034380496|[0.062104,0.06226...|
 0.01691020948937858|-0.001535605|[0.057964,0.05861...|
0.015179627271763176|-0.010175614|[0.055977,0.05669...|
0.017052440941745736| 0.127969792|[0.055921,0.05698...|
 0.01915274320746398| -0.06150692|[0.055353,0.06243...|
0.015159019445515374|-0.025669033|[0.062384,0.06305...|
0.016912732960521242| 0.026608044|[0.058531,0.05968...|
0.015998245081072497|-0.016277919|[0.057086,0.05892...|
0.017255898986918258|-0.004683191|[0.05861,0.058866...|
0.017433904559166824| 0.016468292|[0.05764,0.058828...|
 0.01712202380007528| 0.010852421|[0.057377,0.05972...|
0.017403371946624505|-0.029290548|[0.058315,0.06063...|
0.016814181734545592|-0.039696684|[0.05897,0.059521...|
0.016381890157451607|-0.025581311|[0.057259,0.05773...|
0.016389861892348926| -0.03461797|[0.054941,0.05608...|
0.016632715640287638|-0.004912769|[0.053528,0.05649...|
 0.0172074826766403| 0.048670502|[0.051699,0.05240...|
0.018304816011373216| 0.007877372|[0.051433,0.05402...|
0.017863369347821394|-0.012486897|[0.05394,0.055435...|
0.017604116465622065| 0.006592424|[0.054381,0.05475...|
0.017290287503200087|-0.002812107|[0.053678,0.05465...|
 0.0171773324120682|-0.004508349|[0.054052,0.05543...|
 0.01745631205092968| 0.155226718|[0.053902,0.05446...|
 0.0168815657691438|-0.069725422|[0.053655,0.07011...|
0.015456835954448044|-0.032255827|[0.061968,0.06224...|
0.017115496958651735| 0.019371371|[0.057658,0.05948...|
+--------------------+------------+--------------------+

R^2 = -0.0312873